In [17]:
trafficPath = "file:///home/ubuntu/Downloads/SparkBasic/Dodgers.data"
gamesPath = "file:////home/ubuntu/Downloads/SparkBasic/Dodgers.events"

traffic = sc.textFile(trafficPath)
games = sc.textFile(gamesPath)

from datetime import datetime
import csv
from io import StringIO

def parseTraffic(row):
    Date_fmt = "%m/%d/%Y %H:%M"
    row = row.split(",")
    row[0] = datetime.strptime(row[0],Date_fmt)
    row[1] = int(row[1])
    return (row[0],row[1])

trafficParshed = traffic.map(parseTraffic)

dailyTrend = trafficParshed.map(lambda i: (i[0].date(),i[1]))\
                            .reduceByKey(lambda x,y:x+y)
    


In [18]:
dailyTrend.sortBy(lambda i:-i[1]).take(10)

[(datetime.date(2005, 7, 28), 7661),
 (datetime.date(2005, 7, 29), 7499),
 (datetime.date(2005, 8, 12), 7287),
 (datetime.date(2005, 7, 27), 7238),
 (datetime.date(2005, 9, 23), 7175),
 (datetime.date(2005, 7, 26), 7163),
 (datetime.date(2005, 5, 20), 7119),
 (datetime.date(2005, 8, 11), 7110),
 (datetime.date(2005, 9, 8), 7107),
 (datetime.date(2005, 9, 7), 7082)]

# Merge 2 pair RDD based on the Keys

In [19]:
games.take(2)

['04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�',
 '04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-1�']

In [20]:
#Joining with Games

In [21]:
def parseGames(row):
    DATE_FMT = "%m/%d/%y"
    row = row.split(",")
    row[0] = datetime.strptime(row[0],DATE_FMT).date()
    return (row[0],row[4])

In [22]:
gamesParsed = games.map(parseGames)

In [23]:
gamesParsed.take(4)

[(datetime.date(2005, 4, 12), 'San Francisco'),
 (datetime.date(2005, 4, 13), 'San Francisco'),
 (datetime.date(2005, 4, 15), 'San Diego'),
 (datetime.date(2005, 4, 16), 'San Diego')]

In [24]:
dailyTrendJOIN = dailyTrend.leftOuterJoin(gamesParsed)

In [25]:
dailyTrendJOIN.take(10)

[(datetime.date(2005, 4, 10), (-288, None)),
 (datetime.date(2005, 4, 11), (5062, None)),
 (datetime.date(2005, 4, 14), (6423, None)),
 (datetime.date(2005, 4, 15), (6459, 'San Diego')),
 (datetime.date(2005, 4, 16), (6002, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 18), (5600, None)),
 (datetime.date(2005, 4, 19), (6049, None)),
 (datetime.date(2005, 4, 21), (5977, None)),
 (datetime.date(2005, 4, 22), (6038, None))]

In [32]:
def checkGameDay(row):
    if row[1][1] == None:
        return (row[0],row[1][1],"Regular Day",row[1][0])
    else:
        return (row[0],row[1][1],"Game Day",row[1][0])

In [33]:
dailyTrendByGames = dailyTrendJOIN.map(checkGameDay)

In [34]:
dailyTrendByGames.take(10)

[(datetime.date(2005, 4, 10), None, 'Regular Day', -288),
 (datetime.date(2005, 4, 11), None, 'Regular Day', 5062),
 (datetime.date(2005, 4, 14), None, 'Regular Day', 6423),
 (datetime.date(2005, 4, 15), 'San Diego', 'Game Day', 6459),
 (datetime.date(2005, 4, 16), 'San Diego', 'Game Day', 6002),
 (datetime.date(2005, 4, 17), 'San Diego', 'Game Day', 5322),
 (datetime.date(2005, 4, 18), None, 'Regular Day', 5600),
 (datetime.date(2005, 4, 19), None, 'Regular Day', 6049),
 (datetime.date(2005, 4, 21), None, 'Regular Day', 5977),
 (datetime.date(2005, 4, 22), None, 'Regular Day', 6038)]

In [35]:
dailyTrendByGames.sortBy(lambda i:-i[3]).take(10)

[(datetime.date(2005, 7, 28), 'Cincinnati', 'Game Day', 7661),
 (datetime.date(2005, 7, 29), 'St. Louis', 'Game Day', 7499),
 (datetime.date(2005, 8, 12), 'NY Mets', 'Game Day', 7287),
 (datetime.date(2005, 7, 27), 'Cincinnati', 'Game Day', 7238),
 (datetime.date(2005, 9, 23), 'Pittsburgh', 'Game Day', 7175),
 (datetime.date(2005, 7, 26), 'Cincinnati', 'Game Day', 7163),
 (datetime.date(2005, 5, 20), 'LA Angels', 'Game Day', 7119),
 (datetime.date(2005, 8, 11), 'Philadelphia', 'Game Day', 7110),
 (datetime.date(2005, 9, 8), None, 'Regular Day', 7107),
 (datetime.date(2005, 9, 7), 'San Francisco', 'Game Day', 7082)]

# combineByKey

In [36]:
#to find out average traffic on Game day / regular day

In [42]:
dailyTrendByGames.map(lambda i:(i[2],i[3]))\
                    .combineByKey(lambda value : (value,1),\
                    lambda acc, value:(acc[0]+value,acc[1]+1),\
                    lambda acc1,acc2:(acc1[0]+acc2[0],acc2[1]+acc2[1]))\
                    .mapValues(lambda i:i[0]/i[1])\
                    .collect()

[('Regular Day', 5651.833333333333), ('Game Day', 4633.048076923077)]